<a href="https://colab.research.google.com/github/ambideXtrous9/600-DSA-Problems/blob/main/Context_Find_Using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [104]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [105]:
df = df.iloc[:50000]

In [106]:
# vectorize text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['question'])

# train KNN model with cosine similarity metric
k = 1
knn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='cosine')
knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=1)

# **Accuaracy**

In [107]:
def predict_context(question):
    question_vect = vectorizer.transform([question])
    distances, indices = knn.kneighbors(question_vect)
    top_k_contexts = df['paragraph'].iloc[indices[0]]
    avg_similarities = top_k_contexts.apply(lambda x: cosine_similarity(question_vect, vectorizer.transform([x]))[0][0]).values
    top_context = top_k_contexts.iloc[avg_similarities.argmax()]
    return top_context

In [108]:
def pred(idx):
    ques = df.iloc[idx]['question']
    print("Q : ", ques)
    predicted_context = predict_context(ques)
    print("Actual Context : ", df.iloc[idx]['paragraph'])
    print("Predicted Context : ", predicted_context)

In [109]:
predict = df['question'].apply(predict_context)

In [110]:
res = (predict == df['paragraph']).value_counts()
res

True     48981
False     1019
dtype: int64

In [111]:
accuracy = (res[True] / len(predict))*100
accuracy

97.962